# Importing the necessary libraries

In [1]:
import json
import time
import threading
import uuid
import os
import sys

In [2]:

# Creates a unique file name using timestamp to filename

def get_file_time():
    import uuid
    uniq_append_string = uuid.uuid4().hex
    return "LOCAL_STORAGE_{}".format(uniq_append_string)

# Creating a unique file name with datastore location

def get_inst(file_name=None):
    if file_name is None:
        file_name = get_file_time()
    try:
        os.mkdir('C:/tmp')
    except:
        full_file_name = f"{'C:/tmp'}/{file_name+'.txt'}"
    return full_file_name


# Actual DataStore class(DS)

class DS:

    # 'c' is the dictionary in which we store data
    
    def __init__(self, file_desc=None):
        self.__lock = threading.Lock()
        if(file_desc != None and os.path.isfile(file_desc)):
            self.file_desc = file_desc
        else:
            self.file_desc = get_inst()
        self.f = open(self.file_desc, 'w+')
        #
        self.c = dict(self.f.read())
    # for create operation
    # use syntax "create(key_name,value,timeout_value)" timeout is optional you can continue by passing two arguments without timeout

    def create(self, key, value, timeout=0):
        self.__lock.acquire()
        try:
            self.key = key
            self.value = value
            self.timeout = timeout
            if self.key in self.c:
                print("error: this key already exists")  # error message1 for already existing keys
            else:
                if(self.key.isalpha()):
                    
                    # constraints for file size less than 1GB and Jsonobject value less than 16KB
                    
                    if (len(self.c) < (1024*1024*1024) and sys.getsizeof(self.value) <= (16*1024)):
                        if (self.timeout == 0):
                            self.l = [self.value, self.timeout]
                        else:
                            self.l = [self.value, time.time()+self.timeout]
                        # input key_name capped at 32chars
                        
                        if (len(self.key) <= 32):  
                            self.c[self.key] = self.l
                        else:
                            print("error: Key Size must be between 32 character")
                    else:
                        # error message2 limit exceeded
                        print("error: Memory limit exceeded!!")
                else:
                    # error message3 Invalidkey name 
                    print(
                        "error: Invalid key_name!! key_name must contain only alphabets and no special characters or numbers")
        finally:
            self.__lock.release()

    # for read operation
    # use syntax "read(key_name)"

    def read(self, key):
        self.__lock.acquire()
        try:
            self.json = {}
            self.key = key
            if (self.key not in self.c):
                # error message4
                print(
                    "error: given key does not exist in database. Please enter a valid key")
            else:
                self.b = self.c[key]
                if (self.b[1] != 0):
                    # comparing the present time with expiry time
                    if (time.time() < self.b[1]):
                        # to return the value in the format of JsonObject i.e.,"key_name:value"
                        if(type(self.b[0]) != dict()):
                            res = dict(self.b[0])

                        return res
                    else:
                        print("error: time-to-live of", self.key,"has expired")  # error message5
                else:
                    if(type(self.b[0]) != dict()):
                        res = dict(self.b[0])

                    return res
        finally:
            self.__lock.release()

    # for delete operation
    # use syntax "delete(key_name)"

    def delete(self, key):
        self.key = key
        if (self.key not in self.c):
            # error message4
            raise "error: given key does not exist in database. Please enter a valid key"
        else:
            self.b = self.c[key]
            if (self.b[1] != 0):
                # comparing the current time with expiry time
                if (time.time() < self.b[1]):
                    del self.c[key]
                    print("key is successfully deleted")
                else:
                    print("error: time-to-live of", self.key,"has expired")  # error message5
            else:
                del self.c[key]
                print("key is successfully deleted")
    # it Will save the data in a logical file on path : C:/tmp/*.txt

    def save(self):
        x = json.dumps(self.c, indent=3)
        self.f.write(x)
        self.f.close()

# Creating instance and keyvalue pair

In [3]:
#creating instance for datastore class
ds = DS()

In [4]:
#Creating a key, value with no time to live prop
ds.create('Employee',{'name':'Akash','emproll':1})

In [5]:
#Creating a key, value with time to live prop of 120s
ds.create('Employeefiftyfour',{'name':'Piyush','emproll':54},120)

In [6]:
#Creating a key, value with time to live prop of 150s
ds.create("Employeethree",{'name':"Chitransh",'age':25}, 150)

In [7]:
#Creating a key, value with time to live prop of 3600s
ds.create("HR",{'name':"Shivani",'age':30},3600)

In [8]:
#Creating a key, value with no time to live prop
ds.create("Boss",{'name':"Daksh",'age':32})

# Reading the employees data

In [9]:
ds.read('Employee')

{'name': 'Akash', 'emproll': 1}

In [10]:
ds.read('Employeefiftyfour')

{'name': 'Piyush', 'emproll': 54}

In [11]:
ds.read('HR')

{'name': 'Shivani', 'age': 30}

In [12]:
ds.read('Boss')

{'name': 'Daksh', 'age': 32}

In [13]:
ds.delete('Employeefiftyfour')

key is successfully deleted


In [14]:
ds.read('Employeefiftyfour')

error: given key does not exist in database. Please enter a valid key


In [15]:
ds.read('Employeethree')

{'name': 'Chitransh', 'age': 25}

# Checking for errors

In [16]:
ds.create("Employee_123",{'name':"Chitransh",'age':25}, 150)

error: Invalid key_name!! key_name must contain only alphabets and no special characters or numbers


In [17]:
ds.create('abcdsjdhfsjdhdfjdhfhdhfdhfhdfjskfhskfhsfhsdfhsnsndjfsdjkhfsdhfdfhdfsdkjfhskdfhsdhfsdkfh',{'name': 'Omprakash', 'employeeno': 50})

error: Key Size must be between 32 character


### The code also returns other errors like
1) "invalidkey" if key_length is greater than 32 or key_name contains any numeric,special characters etc.

2) "key doesnot exist" if key_name was mis-spelt or deleted earlier

3) "File memory limit reached" if file memory exceeds 1GB

### This method will store the data in a unique file (C:/tmp/.txt)*

In [18]:
ds.save()

### Checking for Thread safety

In [19]:
t1=threading.Thread(target=(ds.create or ds.read or ds.delete),args=('employeefirst',{'name':'Vaibhav'})) #as per the operation
t2=threading.Thread(target=(ds.create or ds.read or ds.delete),args=('employeesecond',{'employeeno':'2'})) #as per the operation
# starting thread 1 
t1.start() 
# starting thread 2 
t2.start() 
# wait until thread 1 is completely executed 
t1.join() 
# wait until thread 2 is completely executed 
t2.join() 
  
# both threads completely executed 
print("Works Great!!!")

Works Great!!!
